In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv("mental_health_dataset.csv")
df.head()


,Patient_ID,Age,Gender,Sleep_Hours,Stress_Level,Anxiety_Score,Depression_Score,Heart_Rate,Suicidal_Thoughts,Diagnosis
0,P0001,56,Male,5.2,9,9,4,76,0,Anxiety
1,P0002,46,Female,5.3,3,2,6,97,0,NaN
2,P0003,32,Male,5.8,1,8,5,73,0,NaN
3,P0004,60,Other,8.0,7,5,7,82,0,Stress
4,P0005,25,Male,4.4,10,2,10,84,0,Stress


In [6]:
# Encode categorical variables
label_encoder = LabelEncoder()
df["Gender"] = label_encoder.fit_transform(df["Gender"])
df["Diagnosis"] = label_encoder.fit_transform(df["Diagnosis"])
df.head()


,Patient_ID,Age,Gender,Sleep_Hours,Stress_Level,Anxiety_Score,Depression_Score,Heart_Rate,Suicidal_Thoughts,Diagnosis
0,P0001,56,1,5.2,9,9,4,76,0,0
1,P0002,46,0,5.3,3,2,6,97,0,4
2,P0003,32,1,5.8,1,8,5,73,0,4
3,P0004,60,2,8.0,7,5,7,82,0,3
4,P0005,25,1,4.4,10,2,10,84,0,3
